# Google Cloud Vertex AI Search Check Grounding API

> The [Vertex AI Check Grounding API](https://cloud.google.com/generative-ai-app-builder/docs/check-grounding) is one of the standalone APIs in [Vertex AI Agent Builder](https://cloud.google.com/generative-ai-app-builder/docs/builder-apis). It is used to determine how grounded a piece of text (called an answer candidate) is in a given set of reference texts (called facts).

> The Check Grounding API returns an overall support score of 0 to 1, which indicates how much the answer candidate agrees with the given facts. The response also includes citations to the facts supporting each claim in the answer candidate.

> You can use the Check Grounding API for checking any piece of text. It could be a human-generated blurb or a machine-generated response. A typical use case would be to check an LLM-generated response with respect to a given set of facts. Among other things, the citations generated by the API would help distinguish hallucinated claim in the response from grounded claims.

> For more information, see [Check Grounding](https://cloud.google.com/generative-ai-app-builder/docs/check-grounding).

In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-google-community langchain-google-vertexai langchain-chroma langchain-text-splitters google-cloud-discoveryengine langchain-google-community['vertexaisearch'] rich

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# @title # Initialize GCP project for Vertex AI
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

### Load and Prepare data

For this example, we will be using the [Google Wiki page](https://en.wikipedia.org/wiki/Google) to demonstrate how the Vertex Check Grounding API works.

The embeddings are created using the [Vertex AI Embeddings API](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#supported_models) model - `textembedding-gecko@003`

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

vectordb = None
# Load wiki page
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Google")
data = loader.load()

# Split doc into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=5)
splits = text_splitter.split_documents(data)

print(f"Your {len(data)} documents have been split into {len(splits)} chunks")

if vectordb is not None:  # delete existing vectordb if it already exists
    vectordb.delete_collection()

embedding = VertexAIEmbeddings(model_name="textembedding-gecko@003")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

Your 1 documents have been split into 269 chunks


### Testing out the Vertex AI Check Grounding API

Let's setup a standard RAG pipeline of `query -> vector db -> retrieved documents -> Vertex AI LLM` to generate the answer. After the answer is generated from the LLM, pass this `answer_candidate` and the `retrieved_docs` from `vectordb` as `facts` to check how grounded the response from the LLM is.

In [ ]:
from typing import List

from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnableParallel,
    RunnablePassthrough,
    chain,
)
from langchain_google_community import VertexAICheckGroundingWrapper
from langchain_google_vertexai import VertexAI
from rich import print

llm = VertexAI(model_name="gemini-1.0-pro-001", max_output_tokens=1024)

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

template = """Answer the question based only on the following context:
              {context}

              Question:
              {query} """

prompt = PromptTemplate.from_template(template)

output_parser = VertexAICheckGroundingWrapper(
    project_id=PROJECT_ID,
    location_id="global",
    grounding_config="default_grounding_config",
)


@chain
def check_grounding_output_parser(answer_candidate: str, documents: List[Document]):
    return output_parser.with_config(configurable={"documents": documents}).invoke(
        answer_candidate
    )


setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)
create_answer = prompt | llm


@chain
def qa_with_check_grounding(query):
    docs = setup_and_retrieval.invoke(query)
    answer_candidate = create_answer.invoke(docs)
    check_grounding_output = check_grounding_output_parser.invoke(
        answer_candidate, documents=docs["context"]
    )
    return check_grounding_output

### Running the chain with Check Grounding API to verify the grounding

As you can see from the `result`, we obtain the `support_score` of `~0.99` which indicates that LLM response is highly grounded.

The `result` also shows the individual chunks that were used to verify the the various sentences in the `answer_candidate`. Even though we pass 5 facts to the Check Grounding API, only two `cited_chunks` are actually used in the answer generation.

The `results` contains a final `answer_with_citations` that formats each claim with the respective `cited_chunks` that can be referred to easily.

Notice that the second sentence does not contain any citation and hence is `ungrounded`. In your RAG pipeline, you can choose to remove this ungrounded sentence from the final answer that is shown to the user.

In [ ]:
result = qa_with_check_grounding.invoke("how did the name google originate?")
print(result)

CheckGroundingResponse(
    support_score=0.9919261932373047,
    cited_chunks=[
        {
            'chunk_text': 'The name "Google" originated from a misspelling of "googol",[213][214] which refers to 
the number represented by a 1 followed by one-hundred zeros. Page and Brin write in their original paper on 
PageRank:[33] "We chose our system name, Google, because it is a common spelling of googol, or 10100[,] and fits 
well with our goal of building very large-scale search engines." Having found its way increasingly into everyday 
language, the verb "google" was added to the Merriam Webster Collegiate Dictionary and the Oxford English 
Dictionary in 2006, meaning "to use the Google search engine to obtain information on the Internet."[215][216] 
Google\'s mission statement, from the outset, was "to organize the world\'s information and make it universally 
accessible and useful",[217] and its unofficial',
            'source': Document(
                page_content='The name "Google" originated from a misspelling of "googol",[213][214] which refers 
to the number represented by a 1 followed by one-hundred zeros. Page and Brin write in their original paper on 
PageRank:[33] "We chose our system name, Google, because it is a common spelling of googol, or 10100[,] and fits 
well with our goal of building very large-scale search engines." Having found its way increasingly into everyday 
language, the verb "google" was added to the Merriam Webster Collegiate Dictionary and the Oxford English 
Dictionary in 2006, meaning "to use the Google search engine to obtain information on the Internet."[215][216] 
Google\'s mission statement, from the outset, was "to organize the world\'s information and make it universally 
accessible and useful",[217] and its unofficial',
                metadata={
                    'language': 'en',
                    'source': 'https://en.wikipedia.org/wiki/Google',
                    'title': 'Google - Wikipedia'
                }
            )
        },
        {
            'chunk_text': 'Eventually, they changed the name to Google; the name of the search engine was a 
misspelling of the word googol,[21][36][37] a very large number written 10100 (1 followed by 100 zeros), picked to 
signify that the search engine was intended to provide large quantities of information.[38]',
            'source': Document(
                page_content='Eventually, they changed the name to Google; the name of the search engine was a 
misspelling of the word googol,[21][36][37] a very large number written 10100 (1 followed by 100 zeros), picked to 
signify that the search engine was intended to provide large quantities of information.[38]',
                metadata={
                    'language': 'en',
                    'source': 'https://en.wikipedia.org/wiki/Google',
                    'title': 'Google - Wikipedia'
                }
            )
        }
    ],
    claims=[
        {
            'start_pos': 0,
            'end_pos': 137,
            'claim_text': 'The name "Google" originated from a misspelling of "googol", which refers to the number 
represented by a 1 followed by one-hundred zeros.',
            'citation_indices': [0, 1]
        }
    ],
    answer_with_citations='The name "Google" originated from a misspelling of "googol", which refers to the number 
represented by a 1 followed by one-hundred zeros.[0][1]'
)